In [ ]:
'''[Webscraping data from real estate website]
To make sure the code works, please use this link:
http://www.pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/
'''
import requests #fsd
from bs4 import BeautifulSoup
import pandas

r = requests.get("http://www.pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/")
c = r.content
soup = BeautifulSoup(c, "html.parser")   #default parser 
#print(soup.prettify())

# find number of pages
max_page = soup.find_all("a", {"class":"Page"})[-1].text
print( str(max_page) + " pages in total")

base_url = "http://www.pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="

#all[0].find("h4",{"class":"propPrice"}).text.replace("\n","").replace(" ","") #extract the first property's price as text

propertylist=[]  #empty list for storing dictionaries of house info
for page in (0, int(max_page)*10,10 ):
    r = requests.get(base_url + str(page)+".html")
    c = r.content
    soup = BeautifulSoup(c, "html.parser")
    all = soup.find_all("div", {"class":"propertyRow"})  # subtract info that contains properties
    for p in all:
        d = {} #each property will be put in a dict
        try:
            d['Price']=p.find("h4",{"class":"propPrice"}).text.replace("\n","").replace(" ","")
        except:
            d['Price']=None

        try:
            d['Address1'] = p.find_all("span",{"class":"propAddressCollapse"})[0].text
        except:
            d['Address1'] = None

        try:
            d['Address2'] = p.find_all("span",{"class":"propAddressCollapse"})[1].text
        except:
            d['Address2'] = None
        try:
            d['Area'] = p.find("span",{"class":"infoSqFt"}).find("b").text

        except:
            d['Area'] = None
        try:
            d['Beds'] = p.find("span",{"class":"infoBed"}).find("b").text

        except:
            d['Beds'] = None
        try:
            mls = p.find("div", {"class":"propertyMLS"}).text
            d['MLS']  = ''.join(c for c in mls if c in '0123456789') #extract only numbers 
        except:
            d['MLS'] = None 

        propertylist.append(d)

df = pandas.DataFrame(propertylist)  #transform the list to data format

df
#df.to_csv("properties.csv")